In [1]:
# Data handling libraries
import pandas as pd
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
import numpy as np
import boto3
import io
import requests as req

# File manipulation libraries
import os
import pickle

# Data visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline

# Initialize S3 client, location of files for this project
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/blog_data/GHG-GDP_Divergence_D3/"

RAW_DATA = s3_folder + "Raw_Data/"
PROCESSED_DATA = s3_folder + "Processed_Data/"
FINAL_DATA = s3_folder + "Final_Data/"
CONVERSIONS = s3_folder + "Conversions/"

# Functions for reading and uploading data to/from S3
def read_from_S3(bucket, key, index_col=0):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=[index_col], encoding="utf8")
    return(df)

def write_to_S3(df, bucket, key):
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer)
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())

Load Raw Data from S3

In [2]:
# These four files are derived from the original CDIAC data sheet
# They were initially cleaned (using code outlined at the bottom of this notebook)
# And then uploaded to Amazon S3
file_names = ['Territorial Emissions GCB', 
              'Consumption Emissions GCB', 
              'Emissions Transfers GCB', 
              'Territorial Emissions CDIAC']

# Initialize a dictionary to store the raw data
cdiac_raw_data = {}

# Load each of the raw datasets from S3
# Reference: https://stackoverflow.com/questions/37703634/how-to-import-a-text-file-on-aws-s3-into-pandas-without-writing-to-disk
for file in file_names:
    cdiac_raw_data[file] = read_from_S3(s3_bucket, RAW_DATA+file+".csv")

In [144]:
#cdiac_raw_data["Territorial Emissions GCB"].head()
#cdiac_raw_data["Consumption Emissions GCB"].head()
#cdiac_raw_data["Emissions Transfers GCB"].head()
#cdiac_raw_data["Territorial Emissions CDIAC"].head()

Convert raw data to pct_change data for territorial and consumption emissions, load to S3

In [14]:
def add_summary_range(country,row,summ_type,default_start,default_end):
    if summ_type == 'absolute':
        if country in adjustments:
            val = row[adjustments[country]['end']]-row[adjustments[country]['start']]
            time = '{}-{}'.format(adjustments[country]['start'],adjustments[country]['end'])
        else:
            val = row[default_end]-row[default_start]
            time = '{}-{}'.format(default_start, default_end)
    else:
        if country in adjustments:
            val = row[adjustments[country]['end']]/row[adjustments[country]['start']] - 1
            time = '{}-{}'.format(adjustments[country]['start'],adjustments[country]['end'])
        else:
            val = row[default_end]/row[default_start] - 1
            time = '{}-{}'.format(default_start, default_end)
            
    return(val, time)

adjustments = {
    'Eritrea':{
        'start':2000,
        'end':2011
    },
    'Maldives':{
        'start':2001,
        'end':2015
    },
    'Venezuela, RB':{
        'start':2000,
        'end':2014
    },
    'Bermuda':{
        'start':2000,
        'end':2013
    },
    'Libya':{
        'start':2000,
        'end':2011
    }
}

# Territory data
territorial_emissions_abs_raw = cdiac_raw_data["Territorial Emissions GCB"]

territory_gcb_abs_val = territorial_emissions_abs_raw.loc[2000:2015].transpose()
territory_gcb_abs_val['Summary Range'], territory_gcb_abs_val["Summary Range Years"] = \
zip(*territory_gcb_abs_val.apply(lambda row: add_summary_range(row.name,row,'absolute',2000,2015),axis=1))

territory_gcb_pct_change = territorial_emissions_abs_raw.loc[1999:2015].transpose().pct_change(axis=1).loc[:,2000:]
territory_gcb_pct_change['Summary Range'], territory_gcb_pct_change["Summary Range Years"] = \
zip(*territory_gcb_abs_val.apply(lambda row: add_summary_range(row.name,row,'percent',2000,2015),axis=1))

territory_gcb_abs_change = territorial_emissions_abs_raw.loc[1999:2015].transpose().diff(periods=1, axis=1).loc[:,2000:]
territory_gcb_abs_change['Summary Range'], territory_gcb_abs_change["Summary Range Years"] = \
zip(*territory_gcb_abs_val.apply(lambda row: add_summary_range(row.name,row,'absolute',2000,2015),axis=1))

adjustments = {
    'Eritrea':{
        'start':2000,
        'end':2011
    },
    'Maldives':{
        'start':2001,
        'end':2014
    },
    'Venezuela, RB':{
        'start':2000,
        'end':2014
    },
    'Bermuda':{
        'start':2000,
        'end':2013
    },
    'Libya':{
        'start':2000,
        'end':2011
    }
}

# Consumption data
consumption_emissions_abs_raw = cdiac_raw_data["Consumption Emissions GCB"]

consumption_gcb_abs_val = consumption_emissions_abs_raw.loc[2000:2014].transpose()
consumption_gcb_abs_val['Summary Range'], consumption_gcb_abs_val['Summary Range Years'] = \
zip(*consumption_gcb_abs_val.apply(lambda row: add_summary_range(row.name,row,'absolute',2000,2014),axis=1))

consumption_gcb_pct_change = consumption_emissions_abs_raw.loc[1999:2014].transpose().pct_change(axis=1).loc[:,2000:]
consumption_gcb_pct_change['Summary Range'], consumption_gcb_pct_change['Summary Range Years'] = \
zip(*consumption_gcb_abs_val.apply(lambda row: add_summary_range(row.name,row,'percent',2000,2014),axis=1))

consumption_gcb_abs_change = consumption_emissions_abs_raw.loc[1999:2014].transpose().diff(periods=1,axis=1).loc[:,2000:]
consumption_gcb_abs_change['Summary Range'], consumption_gcb_abs_change['Summary Range Years'] = \
zip(*consumption_gcb_abs_val.apply(lambda row: add_summary_range(row.name,row,'absolute',2000,2014),axis=1))



# Upload these percent change figures to S3

write_to_S3(territory_gcb_abs_val, s3_bucket, PROCESSED_DATA + \
                   "Territorial Emissions GCB absolute values 2000-2015.csv")
write_to_S3(territory_gcb_pct_change, s3_bucket, PROCESSED_DATA + \
                   "Territorial Emissions GCB percent changes 2000-2015.csv")
write_to_S3(territory_gcb_abs_change, s3_bucket, PROCESSED_DATA + \
                   "Territorial Emissions GCB absolute changes 2000-2015.csv")

write_to_S3(consumption_gcb_abs_val, s3_bucket, PROCESSED_DATA + \
                   "Consumption Emissions GCB absolute values 2000-2014.csv")
write_to_S3(consumption_gcb_pct_change, s3_bucket, PROCESSED_DATA + \
                   "Consumption Emissions GCB percent changes 2000-2014.csv")
write_to_S3(consumption_gcb_abs_change, s3_bucket, PROCESSED_DATA + \
                   "Consumption Emissions GCB absolute changes 2000-2014.csv")

Download Conversions used to align CDIAC, World Bank, and ISO3 country designations

In [15]:
# CDIAC names to World Bank names
cdiac_to_wb_name_conversion = read_from_S3(s3_bucket, CONVERSIONS+"CDIAC to World Bank name conversion.csv")
# World Bank names to ISO3 codes
wb_name_to_iso3_conversion = read_from_S3(s3_bucket, CONVERSIONS+"World Bank to ISO3 name conversion.csv")

Create final data for the D3 application by adding ISO3 codes to the CDIAC pct change data

In [16]:
# Download pct_change data from S3
territory_gcb_abs_val = read_from_S3(s3_bucket, PROCESSED_DATA+"Territorial Emissions GCB absolute values 2000-2015.csv")
consumption_gcb_abs_val = read_from_S3(s3_bucket, PROCESSED_DATA+"Consumption Emissions GCB absolute values 2000-2015.csv")

territory_gcb_pct_change = read_from_S3(s3_bucket, PROCESSED_DATA+"Territorial Emissions GCB percent changes 2000-2015.csv")
consumption_gcb_pct_change = read_from_S3(s3_bucket, PROCESSED_DATA+"Consumption Emissions GCB percent changes 2000-2015.csv")

territory_gcb_abs_change = read_from_S3(s3_bucket, PROCESSED_DATA+"Territorial Emissions GCB percent changes 2000-2015.csv")
consumption_gcb_abs_change = read_from_S3(s3_bucket, PROCESSED_DATA+"Consumption Emissions GCB percent changes 2000-2015.csv")

dfs = {
    "Production CO2 Emissions Absolute Value":territory_gcb_abs_val,
    "Consumption CO2 Emissions Absolute Value":consumption_gcb_abs_val,
    "Production CO2 Emissions Percent Change":territory_gcb_pct_change,
    "Consumption CO2 Emissions Percent Change":consumption_gcb_pct_change,
    "Production CO2 Emissions Absolute Change":territory_gcb_abs_change,
    "Consumption CO2 Emissions Absolute Change":consumption_gcb_abs_change
}


# Name for Congo didn't match in the CDIAC data and crosswalk file
def replace_congo(name):
    if name == "Congo":
        return("Congo (Rep)")
    else:
        return(name)
    
# Add the wb_name to each dataframe
def fetch_name(name):
    try:
        return(cdiac_to_wb_name_conversion.loc[name][0])
    except:
        return(np.nan)
    
def add_iso(name):
    try:
        return(wb_name_to_iso3_conversion.loc[name,"ISO"])
    except:
        return(np.nan)

for df_name, df in dfs.items():
    print(df_name)
    df.index = list(map(replace_congo, df.index))
    df["Country Name"] = list(map(fetch_name, df.index)) 
    df = df.loc[pd.notnull(df["Country Name"])]
    df = df.set_index("Country Name")
    df["ISO"] = list(map(add_iso, df.index))
    df = df.loc[pd.notnull(df["ISO"])]
    df["Indicator"] = df_name
    dfs[df_name] = df

Production CO2 Emissions Absolute Value
Consumption CO2 Emissions Absolute Value
Production CO2 Emissions Percent Change
Consumption CO2 Emissions Percent Change
Production CO2 Emissions Absolute Change
Consumption CO2 Emissions Absolute Change


In [17]:
# Export final files
write_to_S3(dfs["Production CO2 Emissions Absolute Value"], s3_bucket, FINAL_DATA + "Territory Emissions GCB absolute values with ISO3 2000-2015.csv")
write_to_S3(dfs["Consumption CO2 Emissions Absolute Value"], s3_bucket, FINAL_DATA + "Consumption Emissions GCB absolute values with ISO3 2000-2015.csv")

write_to_S3(dfs["Production CO2 Emissions Percent Change"], s3_bucket, FINAL_DATA + "Territory Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv")
write_to_S3(dfs["Consumption CO2 Emissions Percent Change"], s3_bucket, FINAL_DATA + "Consumption Emissions GCB percent changes with ISO3 2000-2014 plus summary data.csv")

write_to_S3(dfs["Production CO2 Emissions Absolute Change"], s3_bucket, FINAL_DATA + "Territory Emissions GCB absolute changes with ISO3 2000-2015 plus summary data.csv")
write_to_S3(dfs["Consumption CO2 Emissions Absolute Change"], s3_bucket, FINAL_DATA + "Consumption Emissions GCB absolute changes with ISO3 2000-2014 plus summary data.csv")

In [200]:
# Territory or Consumption?
emissions_type = "Territory"
# absolute values or percent changes?
metric = "absolute values"

df = read_from_S3(s3_bucket, FINAL_DATA + \
                  "{} Emissions GCB {} with ISO3 2000-2015 plus summary data.csv".format(emissions_type,metric) \
                  , index_col="ISO")
df

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Summary Range,Summary Range Years,Indicator
ISO,,,,,,,,,,,,,,,,,,,,
ALB,Albania,0.824000,0.879000,1.023000,1.171000,1.136000,1.160000,1.063000,1.071000,1.193000,1.194000,1.254000,1.429000,1.285000,1.313000,1.308100,1.323801,0.499801,2000-2015,Production CO2 Emissions Absolute Value
DZA,Algeria,23.979000,22.987000,24.776000,25.234000,24.405000,29.214000,27.588000,29.860000,30.079000,33.115000,32.500000,33.048000,35.448000,36.601000,39.257647,40.921606,16.942606,2000-2015,Production CO2 Emissions Absolute Value
AND,Andorra,0.143000,0.143000,0.145000,0.146000,0.154000,0.157000,0.149000,0.147000,0.147000,0.141000,0.141000,0.134000,0.134000,0.134000,0.134952,0.135947,-0.007053,2000-2015,Production CO2 Emissions Absolute Value
AGO,Angola,2.602000,2.654000,3.454000,2.472000,5.125000,5.224000,6.072000,6.859000,7.011000,7.579000,7.924000,8.274000,9.108000,8.853000,8.848174,8.990173,6.388173,2000-2015,Production CO2 Emissions Absolute Value
ATG,Antigua and Barbuda,0.094000,0.094000,0.099000,0.106000,0.111000,0.112000,0.116000,0.128000,0.131000,0.139000,0.143000,0.140000,0.143000,0.143000,0.146480,0.149771,0.055771,2000-2015,Production CO2 Emissions Absolute Value
ARG,Argentina,38.761000,36.466000,34.010000,36.832000,42.975000,44.208000,47.842000,47.771000,51.570000,49.076000,51.246000,52.259000,52.457000,51.764000,52.080937,52.799821,14.038821,2000-2015,Production CO2 Emissions Absolute Value
ARM,Armenia,0.945000,0.966000,0.830000,0.935000,0.994000,1.187000,1.195000,1.381000,1.516000,1.189000,1.150000,1.341000,1.553000,1.499000,1.490023,1.525120,0.580120,2000-2015,Production CO2 Emissions Absolute Value
AUS,Australia,95.492749,97.612808,98.761296,100.531107,104.126436,105.235025,106.750573,108.847285,110.326918,111.476113,110.862716,110.181640,110.934183,108.328039,107.294472,109.099849,13.607100,2000-2015,Production CO2 Emissions Absolute Value
AUT,Austria,18.088085,19.186537,19.685205,21.294747,21.394377,21.721753,20.997530,20.269681,20.214589,18.472359,19.795804,19.194096,18.476708,18.547249,17.538909,18.161297,0.073212,2000-2015,Production CO2 Emissions Absolute Value


Process GDP and other WB indicator Data

In [18]:
adjustments = {
    'Eritrea':{
        'start':'2000',
        'end':'2011'
    },
    'Maldives':{
        'start':'2001',
        'end':'2015'
    },
    'Venezuela, RB':{
        'start':'2000',
        'end':'2014'
    },
    'Bermuda':{
        'start':'2000',
        'end':'2013'
    },
    'Libya':{
        'start':'2000',
        'end':'2011'
    }
}

res = req.get("http://api.worldbank.org/countries/all/indicators/NY.GDP.MKTP.KD?date=1999:2016&format=json&per_page=10000")
data = pd.io.json.json_normalize(res.json()[1])
data = data[["country.value", "date", "value"]]
value_name = 'GDP Absolute Value'
data.columns = ["Country Name", "Year", value_name]
data = data.pivot(index="Country Name", columns="Year", values=value_name).astype(float)
data["ISO"] = list(map(add_iso, data.index))
data = data.loc[pd.notnull(data["ISO"])]
data["Indicator"] = value_name 

# Absolute values
data['Summary Range'], data['Summary Range Years'] = zip(*data.apply(lambda row: add_summary_range(row.name,row,'absolute','2000','2015'),axis=1)) 

# Setting up to calculate changes & their summary ranges
wbg_gdp_data = data.copy()
year_cols = [str(yr) for yr in range(1999,2016)]

# Percent changes
data2 = data.loc[:,year_cols].pct_change(axis=1).loc[:,year_cols[1:]]
data2["Indicator"] = "GDP Percent Change"
data2["ISO"] = list(map(add_iso, data2.index))

data2['Summary Range'], data2['Summary Range Years'] = zip(*data.apply(lambda row: add_summary_range(row.name,row,'percent','2000','2015'),axis=1)) 

wbg_gdp_data = wbg_gdp_data.append(data2)
    
# Absolute changes
data3 = data.loc[:,year_cols].diff(periods=1,axis=1).loc[:,year_cols[1:]]
data3["Indicator"] = "GDP Absolute Change"
data3["ISO"] = list(map(add_iso, data3.index))
    
data3['Summary Range'], data3['Summary Range Years'] = zip(*data.apply(lambda row: add_summary_range(row.name,row,'absolute','2000','2015'),axis=1)) 

wbg_gdp_data = wbg_gdp_data.append(data3)

# Clean up    
wbg_gdp_data.index.name = "Country Name"
wbg_gdp_data = wbg_gdp_data.drop(["1999", "2016"], axis=1)

In [19]:
wbg_gdp_data.loc['Libya']

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ISO,Indicator,Summary Range,Summary Range Years
Country Name,,,,,,,,,,,,,,,,,,,,
Libya,4.802634e+10,4.717970e+10,4.672732e+10,5.280934e+10,5.516550e+10,6.171405e+10,6.572580e+10,6.990091e+10,7.176542e+10,7.119837e+10,7.477344e+10,2.835714e+10,NaN,NaN,NaN,NaN,LBY,GDP Absolute Value,-1.966920e+10,2000-2011
Libya,3.679213e-02,-1.762878e-02,-9.588494e-03,1.301600e-01,4.461630e-02,1.187073e-01,6.500547e-02,6.352317e-02,2.667356e-02,-7.901418e-03,5.021290e-02,-6.207592e-01,NaN,NaN,NaN,NaN,LBY,GDP Percent Change,-4.095503e-01,2000-2011
Libya,1.704287e+09,-8.466459e+08,-4.523822e+08,6.082028e+09,2.356158e+09,6.548547e+09,4.011751e+09,4.175111e+09,1.864506e+09,-5.670486e+08,3.575076e+09,-4.641630e+10,NaN,NaN,NaN,NaN,LBY,GDP Absolute Change,-1.966920e+10,2000-2011


In [20]:
write_to_S3(wbg_gdp_data, s3_bucket, FINAL_DATA + "World Bank GDP Data with ISO3, 2000-2015 with Summary Values.csv")

In [21]:
production_co2_emissions_absolute_value = read_from_S3(s3_bucket, FINAL_DATA+"Territory Emissions GCB absolute values with ISO3 2000-2015.csv")
consumption_co2_emissions_absolute_value = read_from_S3(s3_bucket, FINAL_DATA+"Consumption Emissions GCB absolute values with ISO3 2000-2015.csv")

production_co2_emissions_pct_change = read_from_S3(s3_bucket, FINAL_DATA+"Territory Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv")
consumption_co2_emissions_pct_change = read_from_S3(s3_bucket, FINAL_DATA+"Consumption Emissions GCB percent changes with ISO3 2000-2014 plus summary data.csv")

production_co2_emissions_abs_change = read_from_S3(s3_bucket, FINAL_DATA+"Territory Emissions GCB absolute changes with ISO3 2000-2015 plus summary data.csv")
consumption_co2_emissions_abs_change = read_from_S3(s3_bucket, FINAL_DATA+"Consumption Emissions GCB absolute changes with ISO3 2000-2014 plus summary data.csv")

gdp_data = read_from_S3(s3_bucket, FINAL_DATA + "World Bank GDP Data with ISO3, 2000-2015 with Summary Values.csv")

dsets = [production_co2_emissions_absolute_value, consumption_co2_emissions_absolute_value,
         production_co2_emissions_pct_change, consumption_co2_emissions_pct_change,
         production_co2_emissions_abs_change, consumption_co2_emissions_abs_change,
         gdp_data]

In [22]:
keep_these_countries = production_co2_emissions_pct_change.reset_index().set_index("ISO").index

def make_one_file(dsets):
    df = dsets[0].reset_index().set_index("ISO").loc[keep_these_countries].reset_index()
    print(df.shape)
    for i in range(1, len(dsets)):
        df = df.append(dsets[i].reset_index().set_index("ISO").loc[keep_these_countries].reset_index())
        print(df.shape)
    return(df)

final_data = make_one_file(dsets)

(183, 21)
(366, 21)
(549, 21)
(732, 21)
(915, 21)
(1098, 21)
(1647, 21)


In [23]:
# Write to S3
write_to_S3(final_data, s3_bucket, FINAL_DATA + \
                   "All Data Together.csv")

final_data.to_csv("/Users/nathansuberi/Documents/GitHub/nsuberi.github.io/wri-ghg/final-data/All Data Together Concise.csv")

In [6]:
# Read final data
final_data = read_from_S3(s3_bucket, FINAL_DATA + \
                   "All Data Together.csv")
final_data.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Country Name,ISO,Indicator,Summary Range,Summary Range Years
0,0.824,0.879,1.023,1.171,1.136,1.160,1.063,1.071,1.193,1.194,1.254,1.429,1.285,1.313,1.308100,1.323801,Albania,ALB,Production CO2 Emissions Absolute Value,0.499801,2000-2015
1,23.979,22.987,24.776,25.234,24.405,29.214,27.588,29.860,30.079,33.115,32.500,33.048,35.448,36.601,39.257647,40.921606,Algeria,DZA,Production CO2 Emissions Absolute Value,16.942606,2000-2015
2,0.143,0.143,0.145,0.146,0.154,0.157,0.149,0.147,0.147,0.141,0.141,0.134,0.134,0.134,0.134952,0.135947,Andorra,AND,Production CO2 Emissions Absolute Value,-0.007053,2000-2015
3,2.602,2.654,3.454,2.472,5.125,5.224,6.072,6.859,7.011,7.579,7.924,8.274,9.108,8.853,8.848174,8.990173,Angola,AGO,Production CO2 Emissions Absolute Value,6.388173,2000-2015
4,0.094,0.094,0.099,0.106,0.111,0.112,0.116,0.128,0.131,0.139,0.143,0.140,0.143,0.143,0.146480,0.149771,Antigua and Barbuda,ATG,Production CO2 Emissions Absolute Value,0.055771,2000-2015


In [7]:
final_data['Indicator'].unique()

array(['Production CO2 Emissions Absolute Value',
       'Consumption CO2 Emissions Absolute Value', nan,
       'Production CO2 Emissions Percent Change',
       'Consumption CO2 Emissions Percent Change',
       'Production CO2 Emissions Absolute Change',
       'Consumption CO2 Emissions Absolute Change', 'GDP Absolute Value',
       'GDP Percent Change', 'GDP Absolute Change'], dtype=object)

In [18]:
import math

countries = final_data['ISO'].unique()
cntry_data = final_data.set_index('ISO')
for cntry in countries:
    data = cntry_data.loc[cntry]
    x = data.loc[data['Indicator']=='Production CO2 Emissions Percent Change', 'Summary Range'].values
    y = data.loc[data['Indicator']=='GDP Percent Change', 'Summary Range'].values
    print(x,y)
    deg = math.atan(y/x)*180/math.pi
    
    if(x>0):
        if(deg>=-45):
            deg = deg - 45
        else:
            deg = -135 - deg
        
    else:
        deg = -1*deg
        if(deg<=45):
            deg = deg + 45
        else:
            deg = 135 - deg
        
    print(cntry, deg)

[ 0.60655503] [ 0.80493138]
ALB 8.000194039993765
[ 0.70656017] [ 0.71858514]
DZA 0.48343404923929256
[-0.04931959] [ 0.22125593]
AND 57.566213048230594
[ 2.45510098] [ 2.00908822]
AGO -5.7054318693906865
[ 0.59331214] [ 0.27428638]
ATG -20.189044409357702
[ 0.36218934] [ 0.50200817]
ARG 9.190268843467216
[ 0.61388324] [ 1.66292521]
ARM 24.737938524422304
[ 0.14249354] [ 0.54310439]
AUS 30.29873917211559
[ 0.00404754] [ 0.22370868]
AUT 43.963466585089904
[ 0.30012825] [ 3.48979911]
AZE 40.08456696879668
[ 0.94988671] [ 0.05913363]
BHS -41.437742563474345
[ 0.77899581] [ 1.01747939]
BHR 7.561849151106628
[ 1.74554027] [ 1.33728481]
BGD -7.543728253334571
[ 0.2737368] [ 0.11947036]
BRB -21.421501049337483
[ 0.13131543] [ 1.16631544]
BLR 38.57612217047033
[-0.20873992] [ 0.23388059]
BEL 86.74911531779301
[ 0.30698464] [ 0.68280292]
BLZ 20.791564125564392
[ 2.67678957] [ 0.83677657]
BEN -27.640518721281175
[-0.22047244] [ 0.04155454]
BMU 55.67386189398175
[ 1.36296361] [ 2.00464604]
BTN 10

In [267]:
indicators = final_data["Indicator"].unique()
year_cols = [str(yr) for yr in range(2000,2016)]
extents = {}
long_extents = {}
for indicator in indicators:
    data = final_data.loc[final_data["Indicator"]==indicator, year_cols]
    extents[indicator] = [data.min().min(), data.max().max()]
    
    summary_data = final_data.loc[final_data["Indicator"]==indicator, "Summary Range"]
    long_extents[indicator] = [summary_data.min(), summary_data.max()]

In [268]:
final_data[pd.isnull(final_data["Summary Range"])]

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Country Name,ISO,Indicator,Summary Range,Summary Range Years
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andorra,AND,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Angola,AGO,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Antigua and Barbuda,ATG,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bahamas, The",BHS,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Barbados,BRB,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Belize,BLZ,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bermuda,BMU,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bhutan,BTN,Consumption CO2 Emissions Absolute Value,NaN,2000-2014
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bosnia and Herzegovina,BIH,Consumption CO2 Emissions Absolute Value,NaN,2000-2014


In [ ]:
final_data

In [269]:
extents

{'Production CO2 Emissions Absolute Value': [0.0069999999999999993,
  9896.7060457050302],
 'Consumption CO2 Emissions Absolute Value': [-0.31450916878671931,
  2470.0722301013307],
 nan: [nan, nan],
 'Production CO2 Emissions Percent Change': [-0.58231707317073167,
  5.8064516129032278],
 'Consumption CO2 Emissions Percent Change': [-2.2820333549161385,
  4.0330301875807812],
 'Production CO2 Emissions Absolute Change': [-0.58231707317073167,
  5.8064516129032278],
 'Consumption CO2 Emissions Absolute Change': [-2.2820333549161385,
  4.0330301875807812],
 'GDP Absolute Value': [118428316.402523, 75636751196801.094],
 'GDP Percent Change': [-0.62075919584900086, 1.7918065792259044],
 'GDP Absolute Change': [-1118392942734.1953, 2727043480126.0]}

In [270]:
long_extents

{'Production CO2 Emissions Absolute Value': [-157.98880529674875,
  3111.6066568448969],
 'Consumption CO2 Emissions Absolute Value': [-97.065613082643722,
  1600.804809484755],
 nan: [nan, nan],
 'Production CO2 Emissions Percent Change': [-0.36313606588722158,
  13.091579476991498],
 'Consumption CO2 Emissions Percent Change': [-0.39043823222336332,
  5.0366677469343344],
 'Production CO2 Emissions Absolute Change': [-0.36313606588722158,
  13.091579476991498],
 'Consumption CO2 Emissions Absolute Change': [-0.39043823222336332,
  5.0366677469343344],
 'GDP Absolute Value': [-19669203139.392899, 25635055621226.297],
 'GDP Percent Change': [-0.40955028618152017, 4.2559671805372483],
 'GDP Absolute Change': [-19669203139.392899, 25635055621226.297]}

In [271]:
final_data[(final_data['Country Name']=='Eritrea') & ['GDP' in ind if type(ind)==str else False for ind in final_data['Indicator']]]

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Country Name,ISO,Indicator,Summary Range,Summary Range Years
156,1.939336e+09,2.109133e+09,2.172522e+09,2.114830e+09,2.145532e+09,2.200768e+09,2.179437e+09,2.210534e+09,1.994277e+09,2.071585e+09,2.117040e+09,2.300794e+09,NaN,NaN,NaN,NaN,Eritrea,ERI,GDP Absolute Value,3.614583e+08,2000-2011
157,-3.141986e-02,8.755435e-02,3.005429e-02,-2.655516e-02,1.451736e-02,2.574450e-02,-9.692169e-03,1.426822e-02,-9.783030e-02,3.876502e-02,2.194191e-02,8.679800e-02,NaN,NaN,NaN,NaN,Eritrea,ERI,GDP Percent Change,1.863825e-01,2000-2011
158,-6.291029e+07,1.697973e+08,6.338851e+07,-5.769166e+07,3.070174e+07,5.523564e+07,-2.133021e+07,3.109669e+07,-2.162572e+08,7.730817e+07,4.545452e+07,1.837548e+08,NaN,NaN,NaN,NaN,Eritrea,ERI,GDP Absolute Change,3.614583e+08,2000-2011


In [215]:
production_co2_emissions_absolute_value['ISO'].unique()

array(['ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'AUS', 'AUT',
       'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN',
       'BMU', 'BTN', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA', 'BDI',
       'KHM', 'CAN', 'CPV', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM',
       'COG', 'CRI', 'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'ZAR', 'DNK',
       'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST',
       'ETH', 'FSM', 'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU',
       'GHA', 'GRC', 'GRL', 'GRD', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI',
       'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRQ', 'IRL', 'IRN',
       'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'KWT',
       'KGZ', 'LAO', 'LVA', 'LBN', 'LBR', 'LBY', 'LTU', 'LUX', 'MAC',
       'MKD', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL', 'MRT',
       'MUS', 'MEX', 'MNG', 'MNE', 'MAR', 'MOZ', 'MMR', 'NAM', 'NPL',
       'NLD', 'NZL', 'NIC', 'NER', 'NGA', 'NOR', 'OMN', 'PAK', 'PLW',
       'PAN', 'PNG',

Experimentation Below

In [58]:
data_names_and_codes = {'NY.GDP.MKTP.KD': 'GDP (current US$)'}
# {'EG.ELC.ACCS.ZS': 'Access to electricity (% of population)',
#  'EG.FEC.RNEW.ZS': 'Renewable energy consumption (% of total final energy consumption)',
#  'IT.NET.USER.ZS': 'Individuals using the Internet (% of population)',
#  'NE.CON.PRVT.PC.KD': 'Household final consumption expenditure per capita (constant 2010 US$)',
#  'NV.IND.TOTL.KD': 'Industry, value added (constant 2010 US$)',
#  'NY.GDP.TOTL.RT.ZS': 'Total natural resources rents (% of GDP)',
#  'SG.GEN.PARL.ZS': 'Proportion of seats held by women in national parliaments (%)',
#  'SL.EMP.TOTL.SP.ZS': 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)',
#  'SM.POP.NETM': 'Net migration',
#  'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)',
#  'SP.URB.TOTL.IN.ZS': 'Urban population (% of total)',
#  'TM.VAL.MRCH.CD.WT': 'Merchandise imports (current US$)'}
 

column_long_name_to_short_name = {'GDP (current US$)': 'GDP'}
#     {'Renewable energy consumption (% of total final energy consumption)': 'renewable_energy_consumption_of_total_final_energy_consumpti',
#     'Household final consumption expenditure per capita (constant 2010 US$)': 'household_final_consumption_expenditure_per_capita_constant_20',
#     'Merchandise imports (current US$)': 'merchandise_imports_current_us_tm_val_mrch_cd_wt',
#     'Industry, value added (constant 2010 US$)': 'industry_value_added_constant_2010_us_nv_ind_totl_kd',
#     'Access to electricity (% of population)': 'access_to_electricity_of_population_eg_elc_accs_zs',
#     'Urban population (% of total)': 'urban_population_of_total_sp_urb_totl_in_zs',
#     'Employment to population ratio, 15+, total (%) (modeled ILO estimate)': 'employment_to_population_ratio_15_total_modeled_ilo_est',
#     'Total natural resources rents (% of GDP)': 'total_natural_resources_rents_of_gdp_ny_gdp_totl_rt_zs',
#     'Life expectancy at birth, total (years)': 'life_expectancy_at_birth_total_years_sp_dyn_le00_in',
#     'Net migration': 'net_migration_sm_pop_netm',
#     'Proportion of seats held by women in national parliaments (%)': 'proportion_of_seats_held_by_women_in_national_parliaments',
#     'Individuals using the Internet (% of population)': 'individuals_using_the_internet_of_population_it_net_user_z'}

series_code_to_data_viz_name = {}
for key, value in data_names_and_codes.items():
    series_code_to_data_viz_name[key] = column_long_name_to_short_name[value]
    
series_code_to_data_viz_name 

{'NY.GDP.MKTP.CD': 'GDP'}

In [ ]:
indicators = series_code_to_data_viz_name

for indicator in indicators:
    # Results are paginated
    print(indicator)
    res = req.get("http://api.worldbank.org/countries/all/indicators/{}?date=1999:2016&format=json&per_page=10000".format(indicator))
    data = pd.io.json.json_normalize(res.json()[1])
    data = data[["country.value", "date", "value"]]
    value_name = series_code_to_data_viz_name[indicator]
    data.columns = ["Country Name", "Year", value_name]
    data = data.pivot(index="Country Name", columns="Year", values=value_name).astype(float)
    data["ISO"] = list(map(add_iso, data.index))
    data = data.loc[pd.notnull(data["ISO"])]
    data["Indicator"] = value_name 
    
    all_world_bank_data = all_world_bank_data.append(data)
    
    if indicator == "NY.GDP.MKTP.CD":
        year_cols = [str(yr) for yr in range(1999,2016)]
        data = data.loc[:,year_cols].pct_change(axis=1).loc[:,year_cols[1:]]
        data["Indicator"] = "GDP percent change"
        data["ISO"] = list(map(add_iso, data.index))
        all_world_bank_data = all_world_bank_data.append(data)
    
all_world_bank_data.index.name = "Country Name"
all_world_bank_data = all_world_bank_data.drop(["1999", "2016"], axis=1)

In [26]:
all_world_bank_data.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ISO,Indicator
Country Name,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,2.461666e+09,4.128821e+09,4.583644e+09,5.285466e+09,6.275074e+09,7.057598e+09,9.843842e+09,1.019053e+10,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.004633e+10,2.005019e+10,1.921556e+10,AFG,GDP
Albania,3.632044e+09,4.060759e+09,4.435079e+09,5.746946e+09,7.314865e+09,8.158549e+09,8.992642e+09,1.070101e+10,1.288135e+10,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322824e+10,1.133526e+10,ALB,GDP
Algeria,5.479025e+10,5.474471e+10,5.676029e+10,6.786383e+10,8.532500e+10,1.031982e+11,1.170273e+11,1.349771e+11,1.710007e+11,1.372110e+11,1.612073e+11,2.000191e+11,2.090590e+11,2.097550e+11,2.138100e+11,1.658743e+11,DZA,GDP
American Samoa,NaN,NaN,5.140000e+08,5.270000e+08,5.120000e+08,5.030000e+08,4.960000e+08,5.200000e+08,5.630000e+08,6.780000e+08,5.760000e+08,5.740000e+08,6.440000e+08,6.410000e+08,6.430000e+08,6.590000e+08,ASM,GDP
Andorra,1.434430e+09,1.496913e+09,1.733117e+09,2.398646e+09,2.935659e+09,3.255789e+09,3.543257e+09,4.016972e+09,4.007353e+09,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,AND,GDP


In [29]:
reverse_map = {v: k for k, v in column_long_name_to_short_name.items()}
def create_summary_values_World_Bank(row):
    #print(row)
    if row["Indicator"] == "GDP percent change":
        val = row["2015"] - row["2000"]
        return(val, "2000-2015")
    else:
        indicator = reverse_map[row["Indicator"]]
    
    if indicator == 'Renewable energy consumption (% of total final energy consumption)':
        val = row["2014"] - row["2000"]
        return(val, "2000-2014")

    elif indicator == 'GDP (current US$)':
        val = row["2015"] - row["2000"]
        return(val, "2000-2015")

    elif indicator == 'Household final consumption expenditure per capita (constant 2010 US$)':
        val = row["2015"]
        return(val, "2015")

    elif indicator == 'Merchandise imports (current US$)':
        val = row["2015"] - row["2000"]
        return(val, "2000-2015")

    elif indicator == 'Industry, value added (constant 2010 US$)':
        val = row["2015"] - row["2000"]
        return(val, "2000-2015")

    elif indicator == 'Access to electricity (% of population)':
        val = row["2014"]
        return(val, "2014")

    elif indicator == 'Urban population (% of total)':
        val = row["2015"]
        return(val, "2015")

    elif indicator == 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)':
        val = row["2015"] - row["2000"]
        return(val, "2000-2015")

    elif indicator == 'Total natural resources rents (% of GDP)':
        val = row["2015"]
        return(val, "2015")

    elif indicator == 'Life expectancy at birth, total (years)':
        val = row["2015"]
        return(val, "2015")

    elif indicator == 'Net migration':
        val = row["2012"]
        return(val, "2012")

    elif indicator == 'Proportion of seats held by women in national parliaments (%)':
        val = row["2015"]
        return(val, "2015")

    elif indicator == 'Individuals using the Internet (% of population)':
        val = row["2015"]
        return(val, "2015")
     
summary_data = all_world_bank_data.apply(create_summary_values_World_Bank, axis=1)
all_world_bank_data["Summary Range"], all_world_bank_data["Summary Range Years"] = list(zip(*summary_data))

In [30]:
all_world_bank_data.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ISO,Indicator,Summary Range,Summary Range Years
Country Name,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.163713,1.006099,3.467205,7.209179,13.969172,23.000000,27.506411,34.290512,42.4,47.888466,42.700000,61.514420,69.100000,75.154373,89.5,NaN,AFG,access_to_electricity_of_population_eg_elc_acc...,89.5,2014
Albania,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,ALB,access_to_electricity_of_population_eg_elc_acc...,100.0,2014
Algeria,96.702133,97.004044,97.298698,97.590019,97.881889,98.184265,98.490738,98.806519,99.3,99.443893,99.711174,99.889542,99.973083,99.996918,100.0,NaN,DZA,access_to_electricity_of_population_eg_elc_acc...,100.0,2014
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASM,access_to_electricity_of_population_eg_elc_acc...,NaN,2014
Andorra,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,AND,access_to_electricity_of_population_eg_elc_acc...,100.0,2014


In [31]:
write_to_S3(all_world_bank_data, s3_bucket, FINAL_DATA + "World Bank Data with ISO3, 2000-2015 with Summary Values.csv")

Calculate Index Values

In [32]:
world_bank_data = read_from_S3(s3_bucket, FINAL_DATA + "World Bank Data with ISO3, 2000-2015 with Summary Values.csv")

world_bank_data.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ISO,Indicator,Summary Range,Summary Range Years
Country Name,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.163713,1.006099,3.467205,7.209179,13.969172,23.000000,27.506411,34.290512,42.4,47.888466,42.700000,61.514420,69.100000,75.154373,89.5,NaN,AFG,access_to_electricity_of_population_eg_elc_acc...,89.5,2014
Albania,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,ALB,access_to_electricity_of_population_eg_elc_acc...,100.0,2014
Algeria,96.702133,97.004044,97.298698,97.590019,97.881889,98.184265,98.490738,98.806519,99.3,99.443893,99.711174,99.889542,99.973083,99.996918,100.0,NaN,DZA,access_to_electricity_of_population_eg_elc_acc...,100.0,2014
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASM,access_to_electricity_of_population_eg_elc_acc...,NaN,2014
Andorra,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,AND,access_to_electricity_of_population_eg_elc_acc...,100.0,2014


In [38]:
# Calculating index values
# formula = (1 – ΔCO2)*(1 + ΔGDP) - ΔCO2 + ΔGDP

world_bank_data = read_from_S3(s3_bucket, FINAL_DATA + "World Bank Data with ISO3, 2000-2015 with Summary Values.csv")
gdp_percent_change_data = world_bank_data.set_index("Indicator").loc["GDP percent change"]
gdp_percent_change_data.set_index(["ISO"], inplace=True)
gdp_percent_change_data = gdp_percent_change_data.drop(["Summary Range", "Summary Range Years"], axis=1)

def calc_index(co2, gdp):
    return((1-co2)*(1+gdp) - co2 + gdp)
    
# import CO2 change
## Territorial
territory_emissions = read_from_S3(s3_bucket, FINAL_DATA + \
                   "Territory Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv", index_col = "ISO")

## Consumption
consumption_emissions = read_from_S3(s3_bucket, FINAL_DATA + \
                   "Consumption Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv", index_col = "ISO")

# https://stackoverflow.com/questions/22149584/what-does-axis-in-pandas-mean
country_names_territory = territory_emissions["Country Name"]
country_names_consumption = consumption_emissions["Country Name"]

territory_emissions_gdp_index = calc_index(territory_emissions.drop(["Country Name", "Summary Range", "Summary Range Years", "Indicator"], axis=1), gdp_percent_change_data)
consumption_emissions_gdp_index = calc_index(consumption_emissions.drop(["Country Name", "Summary Range", "Summary Range Years","Indicator"], axis=1), gdp_percent_change_data)

def create_summary_values_ICGGD(row):
    val = row["2015"] - row["2000"]
    return(val, "2000-2015")
    
summary_data = territory_emissions_gdp_index.apply(create_summary_values_ICGGD, axis=1)
territory_emissions_gdp_index["Summary Range"], territory_emissions_gdp_index["Summary Range Years"] = list(zip(*summary_data))

summary_data = consumption_emissions_gdp_index.apply(create_summary_values_ICGGD, axis=1)
consumption_emissions_gdp_index["Summary Range"], consumption_emissions_gdp_index["Summary Range Years"] = list(zip(*summary_data))

territory_emissions_gdp_index["Indicator"] = "ICGGD with Production Emissions"
consumption_emissions_gdp_index["Indicator"] = "ICGGD with Consumption Emissions"
territory_emissions_gdp_index["Country Name"] = country_names_territory
consumption_emissions_gdp_index["Country Name"] = country_names_consumption

In [39]:
# Write to S3
write_to_S3(territory_emissions_gdp_index, s3_bucket, FINAL_DATA + \
                   "ICGGD calculated with Territory Emissions.csv")

write_to_S3(consumption_emissions_gdp_index, s3_bucket, FINAL_DATA + \
                   "ICGGD calculated with Consumption Emissions.csv")

In [76]:
# Create single file that has all the data for the application

#gdp_annual_pct_change = read_from_S3(s3_bucket, FINAL_DATA+".csv")
#gdp_annual_absolute_value = read_from_S3(s3_bucket, FINAL_DATA+".csv")

production_co2_emissions_annual_change = read_from_S3(s3_bucket, FINAL_DATA+"Territory Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv")
production_co2_emissions_absolute_value = read_from_S3(s3_bucket, FINAL_DATA+"Territory Emissions GCB absolute values with ISO3 2000-2015 plus summary data.csv")

consumption_co2_emissions_annual_change = read_from_S3(s3_bucket, FINAL_DATA+"Consumption Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv")
consumption_co2_emissions_absolute_value = read_from_S3(s3_bucket, FINAL_DATA+"Consumption Emissions GCB absolute values with ISO3 2000-2015 plus summary data.csv")

index_with_production_emissions_and_gdp = read_from_S3(s3_bucket, FINAL_DATA+"ICGGD calculated with Territory Emissions.csv")
index_with_consumption_emissions_and_gdp = read_from_S3(s3_bucket, FINAL_DATA+"ICGGD calculated with Consumption Emissions.csv")

world_bank_data = read_from_S3(s3_bucket, FINAL_DATA+"World Bank Data with ISO3, 2000-2015 with Summary Values.csv")

dsets = [production_co2_emissions_annual_change, production_co2_emissions_absolute_value,
        consumption_co2_emissions_annual_change, consumption_co2_emissions_absolute_value,
        index_with_production_emissions_and_gdp, index_with_consumption_emissions_and_gdp,
        world_bank_data]

def print_columns(list_of_dfs):
    for df in list_of_dfs:
        print(df.reset_index().columns)
        
print_columns(dsets)

                         2000      2001      2002      2003      2004  \
Country Name                                                            
Albania              0.012285  0.066748  0.163823  0.144673 -0.029889   
Algeria             -0.045460 -0.041370  0.077827  0.018486 -0.032853   
Andorra              0.021429  0.000000  0.013986  0.006897  0.054795   
Angola               0.042050  0.019985  0.301432 -0.284308  1.073220   
Antigua and Barbuda -0.010526  0.000000  0.053191  0.070707  0.047170   

                         2005      2006      2007      2008      2009  \
Country Name                                                            
Albania              0.021127 -0.083621  0.007526  0.113912  0.000838   
Algeria              0.197050 -0.055658  0.082355  0.007334  0.100934   
Andorra              0.019481 -0.050955 -0.013423  0.000000 -0.040816   
Angola               0.019317  0.162328  0.129611  0.022161  0.081016   
Antigua and Barbuda  0.009009  0.035714  0.103448 

In [53]:
world_bank_data[world_bank_data["Indicator"]=="GDP"]

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ISO,Indicator,Summary Range,Summary Range Years
Country Name,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,2.461666e+09,4.128821e+09,4.583644e+09,5.285466e+09,6.275074e+09,7.057598e+09,9.843842e+09,1.019053e+10,1.248694e+10,1.593680e+10,1.793024e+10,2.053654e+10,2.004633e+10,2.005019e+10,1.970299e+10,AFG,GDP,NaN,2000-2015
Albania,3.632044e+09,4.060759e+09,4.435079e+09,5.746946e+09,7.314865e+09,8.158549e+09,8.992642e+09,1.070101e+10,1.288135e+10,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.278103e+10,1.321986e+10,1.139037e+10,ALB,GDP,7.758321e+09,2000-2015
Algeria,5.479025e+10,5.474471e+10,5.676029e+10,6.786383e+10,8.532500e+10,1.031982e+11,1.170273e+11,1.349771e+11,1.710007e+11,1.372110e+11,1.612073e+11,2.000131e+11,2.090474e+11,2.097835e+11,2.139831e+11,1.647795e+11,DZA,GDP,1.099892e+11,2000-2015
American Samoa,NaN,NaN,5.140000e+08,5.270000e+08,5.120000e+08,5.030000e+08,4.960000e+08,5.200000e+08,5.630000e+08,6.780000e+08,5.760000e+08,5.740000e+08,6.440000e+08,6.390000e+08,6.380000e+08,6.410000e+08,ASM,GDP,NaN,2000-2015
Andorra,1.401695e+09,1.484018e+09,1.717485e+09,2.373928e+09,2.916787e+09,3.248215e+09,3.536633e+09,4.010991e+09,4.001201e+09,3.650083e+09,3.346517e+09,3.427023e+09,3.146152e+09,3.248925e+09,NaN,NaN,AND,GDP,NaN,2000-2015
Angola,9.129595e+09,8.936064e+09,1.249735e+10,1.418895e+10,1.964085e+10,2.823371e+10,4.178948e+10,6.044892e+10,8.417803e+10,7.549238e+10,8.247091e+10,1.041159e+11,1.153984e+11,1.249121e+11,1.267769e+11,1.029622e+11,AGO,GDP,9.383265e+10,2000-2015
Antigua and Barbuda,8.254055e+08,7.959765e+08,8.097545e+08,8.502186e+08,9.137104e+08,1.014980e+09,1.149025e+09,1.302389e+09,1.359734e+09,1.217720e+09,1.147942e+09,1.141865e+09,1.216046e+09,1.195885e+09,1.274330e+09,1.355646e+09,ATG,GDP,5.302404e+08,2000-2015
Argentina,2.842038e+11,2.686968e+11,9.772400e+10,1.275870e+11,1.646579e+11,1.987371e+11,2.325573e+11,2.875305e+11,3.615580e+11,3.329765e+11,4.236274e+11,5.301633e+11,5.459824e+11,5.520251e+11,5.263197e+11,5.847115e+11,ARG,GDP,3.005077e+11,2000-2015
Armenia,1.911564e+09,2.118468e+09,2.376335e+09,2.807061e+09,3.576615e+09,4.900470e+09,6.384452e+09,9.206302e+09,1.166204e+10,8.647937e+09,9.260285e+09,1.014211e+10,1.061932e+10,1.112147e+10,1.160951e+10,1.052918e+10,ARM,GDP,8.617619e+09,2000-2015


In [62]:
consumption_co2_emissions_absolute_value.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ISO,Summary Range,Summary Range Years,Indicator
Country Name,,,,,,,,,,,,,,,,,,,,
Albania,1.029235,1.102482,1.355331,1.506422,1.453613,1.605657,1.516234,1.51755,1.655498,1.696258,1.650886,1.711138,1.598367,1.543575,1.534179,NaN,ALB,0.504944,2000-2014,Consumption CO2 Emissions Absolute Value
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DZA,NaN,2000-2014,Consumption CO2 Emissions Absolute Value
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AND,NaN,2000-2014,Consumption CO2 Emissions Absolute Value
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGO,NaN,2000-2014,Consumption CO2 Emissions Absolute Value
Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATG,NaN,2000-2014,Consumption CO2 Emissions Absolute Value


In [80]:
keep_these_countries = production_co2_emissions_annual_change.reset_index().set_index("ISO").index

def make_one_file(dsets):
    df = dsets[0].reset_index().set_index("ISO").loc[keep_these_countries].reset_index()
    print(df.shape)
    for i in range(1, len(dsets)):
        df = df.append(dsets[i].reset_index().set_index("ISO").loc[keep_these_countries].reset_index())
        print(df.shape)
    return(df)

final_data = make_one_file(dsets)

(183, 21)
(366, 21)
(549, 21)
(732, 21)
(915, 21)
(1098, 21)
(3660, 21)


In [81]:
# Write to S3
write_to_S3(final_data, s3_bucket, FINAL_DATA + \
                   "All Data Together.csv")

final_data.to_csv("/Users/nathansuberi/Desktop/Code Portfolio/nsuberi.github.io/wri-ghg/final-data/All Data Together.csv")

In [ ]:
## Calculate extents in Python... re-did this in javascript

In [73]:
indicators = final_data["Indicator"].unique()
year_cols = [str(yr) for yr in range(2000,2016)]
extents = {}
long_extents = {}
for indicator in indicators:
    data = final_data.loc[final_data["Indicator"]==indicator, year_cols]
    extents[indicator] = [data.min().min(), data.max().max()]
    
    summary_data = final_data.loc[final_data["Indicator"]==indicator, "Summary Range"]
    long_extents[indicator] = [summary_data.min(), summary_data.max()]

In [75]:
print("extents")
print(extents)
print("long extents")
print(long_extents)

extents
{'Production CO2 Emissions Annual Change': [-0.58231707317073167, 5.8064516129032278], 'Production CO2 Emissions Absolute Value': [0.0069999999999999993, 9896.7060457050302], 'Consumption CO2 Emissions Annual Change': [-2.2820333549161385, 4.0330301875807812], 'Consumption CO2 Emissions Absolute Value': [-0.31450916878671931, 2470.0722301013307], 'ICGGD with Production Emissions': [-12.123624635633307, 9.1800704884216824], 'ICGGD with Consumption Emissions': [-7.4118054937117783, 5.9522470514303185], 'access_to_electricity_of_population_eg_elc_accs_zs': [0.0154853165149689, 100.0], 'renewable_energy_consumption_of_total_final_energy_consumpti': [0.0, 98.342609009734005], 'individuals_using_the_internet_of_population_it_net_user_z': [0.000289277, 98.323609649999995], 'household_final_consumption_expenditure_per_capita_constant_20': [152.47162086105499, 48546.572692095098], 'industry_value_added_constant_2010_us_nv_ind_totl_kd': [1322136.39560658, 21258209566653.301], 'total_natu